**Importing Libraries** <br>
Import all the necessary GitHub Repos

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/PaddlePaddle/PaddleOCR


pip intsall all the necessary Dependencies

In [ ]:
!python -m pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install paddleocr

%cd yolov5
!pip install -r requirements.txt

Import all the modules

In [ ]:
import torch
from IPython.display import Image #this is to render predictions

from PIL import Image
import numpy
import timeit
import threading

from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
from matplotlib import pyplot as plt # plot images
import cv2 #opencv
import os # folder directory navigation

import matplotlib
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow
import cv2

**Loading Models**

In [233]:
model_screen = torch.hub.load('/content/yolov5', 'custom', path='./weights/bestm.pt', source='local',_verbose=False)
model_vitals = torch.hub.load('/content/yolov5', 'custom', path='./weights/best_vital.pt', source='local',_verbose=False)
model_hrGraph = torch.hub.load('/content/yolov5', 'custom', path='./weights/best_hr.pt', source='local',_verbose=False)
model_garbage = torch.hub.load('/content/yolov5', 'custom', path='./weights/best_garbage.pt', source='local',_verbose=False)
ocr = PaddleOCR(use_angle_cls=True, lang='en',show_log = False,debug=False)

**Utility Functions**

In [364]:
def removing_multiple_labelling(res,num_classes):
  res_dup = res.xyxy[0]
  arr = []
  for i in range(0,num_classes):
    arr.append([])
  res_list = res_dup.tolist()
  for i in range(0,len(res_list)):
    for j in range(0,num_classes):
      if(int(res_list[i][-1] )== j):
        if(len(arr[j]) == 0):
            arr[j] = res_list[i]
        else:
            if(arr[j][-2] < res_list[i][-2]):
              arr[j] = res_list[i]
  arr_copy = []
  for aj in arr:
    if(len(aj) != 0):
      arr_copy.append(aj)

  arr_copy_tensor = torch.tensor(arr_copy)
  return arr_copy_tensor

dict_vitals = {
    0:"DBP",
    1:"HR",
    2: "MAP",
    3: "RR",
    4: "SBP",
    5: "SPO2"
}

def final_graph_func(image):
  
  img_np = numpy.asarray(image)

  #Using the model weights to detect garbage texts from the image
  results = model_garbage(image)
  res_dup = results.xyxy[0]

  img_gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)

  def get_mask(image,res_dup):
    mask=image
    dh, dw = image.shape
    for dt in res_dup:
      # Split string to float
      x1, y1, x2, y2 = dt[:-2]
      x1=int(x1)
      x2=int(x2)
      y1=int(y1)
      y2=int(y2)
      for j in range(y1,y2):
        for i in range(x1,x2):
          mask[j][i]=0
    return mask

  # Calling the masking function to obtain the image with black masks on the garbage regions
  masked_img=get_mask(img_gray, res_dup)

  # Binarizing the image 
  ret, thresh = cv2.threshold(masked_img, 200, 255, cv2.THRESH_BINARY)
  
  # Post processing of the binarized image to obtain contours and plotting the same.
  def final_processing(image):
    #Extracts all contours
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    edged = cv2.Canny(blurred, 20, 150)
    contours, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  
    data_points = []
    final_img=numpy.full(image.shape,255)
    for contour in contours:
      for data in contour:
        data_points.append((data[0][0],data[0][1]))

    for data in data_points:
      final_img[int(data[1]),int(data[0])]=0

    # Plot the image with grid lines
    fig, ax = plt.subplots()
    ax.imshow(final_img)
    ax.grid(which='both', color='orange', linewidth=0.05)
    ax.plot()
    # Show the plot
    plt.show()

    cv2_imshow(final_img)
    return final_img
  
  final_processing(thresh)

**Screen Extraction using YOLOv5m6**

In [365]:
def screen_extraction(image_path:str):
  im1 = Image.open(image_path)
  results=model_screen(im1, size=640)
  results.xyxy[0] = removing_multiple_labelling(results,1)
  cropped = im1.crop(results.xyxy[0][0][:-2].numpy())
  return cropped

**Vitals Extraction**

In [248]:
def number_extraction(crop):
  result = ocr.ocr(crop, cls=True)
  if(len(result[0]) == 0):
    return -1
  result_list = result[0]
  arr_tuple = []
  for i in result_list:
    arr_tuple.append(i[-1])
  arr_tuple.sort(reverse = True, key = lambda t: t[-1])
  text = arr_tuple[0][0]
  num = ""
  for i in text:
    if i.isdigit():
      num += i
  if num == "":
    return -1
  return int(num)

def vitals_extraction(cropped):
  res=model_vitals(cropped, size=640)
  res_dup = removing_multiple_labelling(res,6)
  ret_dict = {}
  for labels in res_dup:
    cropped_copy = cropped
    crop_save = cropped_copy.crop(labels[:-2].numpy())
    #converting B/W and paddig
    cropped_list = numpy.asarray(crop_save)
    cropped_list=cv2.cvtColor(cropped_list, cv2.COLOR_BGR2GRAY)
    cropped_list = cv2.copyMakeBorder(cropped_list, 50,50,50,50,cv2.BORDER_CONSTANT)
    classes = dict_vitals[int(labels[-1])]
    value = number_extraction(cropped_list)
    if(value != -1 and classes != "MAP"):
      ret_dict[classes] = value
  return ret_dict

**HR Curve Digitization** <br> call graph_extraction() with argument as the image path

In [366]:


# Extracts the graph from the image of the screen to be used as input for the final_graph_func function 
def graph_extraction(image_path:str):
  screen = screen_extraction(image_path)
  res = model_hrGraph(screen, size=640)
  res_dup = removing_multiple_labelling(res,1)
  cropped = screen.crop(res_dup[0][:-2].numpy())
  final_graph_func(cropped)

## FINAL

In [250]:
def inference(image_path:str):
  '''
  Function responsible for inference.
  Args: 
    image_path: str, path to image file. eg. "input/aveksha_micu_mon--209_2023_1_17_12_0_34.jpeg"
  Returns:
    result: dict, final output dictionary. eg. {"HR":"80", "SPO2":"98", "RR":"15", "SBP":"126", "DBP":"86"}
  '''
  result = {}
  screen = screen_extraction(image_path)
  result = vitals_extraction(screen)
  ### put your code here
  
  return result


We used this accuracy funtion to get final Accuracy (same as instructed in FAQ doc)

In [232]:
def accuracy(pred,gt):
  ans = 0.0
  for i in range(0,6):
    if(i == 2):
      continue
    key = dict_vitals[i]
    if((key in gt) and (key in pred)):
      if (gt[key]==pred[key]):
        ans += 0.2
    if((not(key in gt)) and (not(key in pred))):
      ans += 0.2
  return ans

# Inter IIT Tech Meet 11.0
*Team ID: 17*

### Abstract:
Monitoring vitals is an important aspect of providing high-quality care to patients. With the increasing use of technology in healthcare, there are now many digital monitoring systems available that can help to automate the process of tracking vitals, making it more efficient and accurate.

This challenge is aimed at extracting a patient's vitals like SpO2, RR, Systolic Blood Pressure, Diastolic Blood Pressure and MAP from the provided ECG monitor images. We have dealt with this problem in a sequence of three separate subproblems - extracting the screen from the initial image, performing object detection and recognising the text to obtain vitals. We have employed state of the art models, data augmentation techniques, noise reduction models on images, supervised and semi-supervised learning on the data given to obtain a fast and accurate network that takes in an image and outputs the patient's vitals.

### Approach:
We have attempted to solve this problem statement by dividing the core task into several sub tasks and targeting each part as a standalone problem. Our baseline model consists of the following steps :

- **Screen Extraction** - Extracting only the relevant part of the image i.e the monitor. We experimented with several techniques and finally settled with YOLOv5m6.

- **Object Detection to extract vitals** - After experimenting with several architectures we used YOLOv5m6 to identify which number on the screen corresponded to which vital.

- **Character Recognition to obtain measurements** - With the help of Optical Character Recognition(OCR), we read the text contained within the bounding boxes given as output by YOLO.

The above 3 steps lay down the most basic approach we had. Along with this we have later **mentioned our approach for detecting the H.R graph** and digitizing it.

### Dataset :
The dataset we obtained was in three parts:

- **Monitor Segmentation Dataset** - The dataset consists of 2000 images having the segmentation boundaries for the monitors in each image .

- **Classification Dataset** - The second dataset consists of a 1000 monitor images with each monitor belonging to one of the four classes present.

- **Unlabelled Dataset** - The third dataset consists of 7000 unlabelled images with monitors from several different types of classes.

### Baseline Model  :
We train a baseline model to extract the measurements relating to each vital from the monitor image. Our **baseline model** consists of:

- A **ResNet-18** based regressor to obtain the coordinates of the monitors' corners (i.e. to generate a bounding box)

- Followed by **ResNet-18** to perform classification and determine the monitor type.

- This is followed by object detection using **YOLOv5m6** to create bounding boxes around the different vital measurements like Heart Rate, SpO2, RR, MAP etc.

- Finally **Paddle-Paddle OCR** reads the text inside the bounding boxes generated by YOLO and gives the final output.

### Improvements on the Baseline:
We undertook thorough literature reviews and conducted several experiments on our initial baseline by eliminating and adding parts,trying out different models for the subtasks and performing semi-supervised learning. Finally we settled on the following architecture as it gave the best performance.

### Final architecture:
- Screen Extraction via YOLOv5m6
- Vital Sign extraction using YOLOv5m6
- Text recognition using Paddle-Paddle OCR.

### Why did we omit classification?
Since the labeled data given to us was classified into 4 types, we naturally thought of using a CNN based classifier like ResNet to determine the image type and then training a separate YOLO network on each type for higher accuracy. However, since YOLO learns contextual information, it was able to detect the correct vitals irrespective of the monitor type and removal of the classifier lead to a significant decrease in latency while not affecting the accuracy negatively in any way.

### Pipeline :

![Pipeline](./images/flowchart.png)

### Screen Extraction:

- Our first approach to solve this problem was using ResNet-50 to predict the 4 corner points of the quadrilateral tracing the monitor by regression. However, the results were highly inaccurate on unseen data which led us to try out Instance Segmentation of the image by using Mask-RCNN.
 
- Mask-RCNN outputs a bounding box around the monitor along with a pixel by pixel mask. The results after performing semi-supervised learning on the given data were highly accurate. The main issue we faced was an unusually high inference time per image(9 seconds when using Google Colab-CPU).
 
- **YOLO** is extremely fast because it does not deal with complex pipelines. It sees the entire image during training and test time so it implicitly encodes contextual information about classes as well as their appearance. Thus we performed training with YOLOv5m6 and got highly accurate results with a significant improvement in the inference time - **0.6 seconds approximately per image** , 1500% faster than Mask-RCNN.

- We use **YOLOv5 – YOLOv5m6** pretrained on the COCO dataset.

For improving accuracy and a more robust screen extraction, we employ **data augmentations** to learn the given images better.
- We use vertical and horizontal flipping of the images
 
- Changing the image saturation(between -25% and 25%).

- We use Gaussian Blur to blur the images to 6px and also add random noise to 5% of pixels.

![Screen](./images/screen.png)

### YOLO's outputs for screen segmentation.

### Object detection/extracting vitals :
- Our first approach to solve this problem was exploring the different operations available and performing literature reviews of the said options. We read up about FasterRCNN, ResNet and YOLO and tried to understand how these models would analyze our data.

- After thorough reading and reasoning, we chose YOLO to extract vital signs because YOLO can learn contextual information better than other object detection models.It also has the obvious advantages of better speed and accuracy.

### Semi Supervised Learning :
- We were given a small labeled dataset of 1000 images and unlabelled dataset with 7000 images. Thus the most natural thing to do was to carry out semi-supervised learning of YOLO on both the datasets. We chose **Pseudo-Labelling** , a very simple approach to semi supervised learning.

- The model is initially trained on the labeled dataset and is used to make predictions for the unlabeled data. It selects the examples from the unlabeled dataset where the prediction is confident and considers its prediction as pseudo-label. This pseudo labeled dataset is then added to the labeled dataset and the model is then trained again on the expanded labeled dataset.

- We developed a function to manipulate our text files such that we chose only the most confident predictions of each class. We then concatenated these predicted labels to the original dataset and retrained our network thereby implementing SSL with pseudo labeling.

![vitals](./images/vitals.png)

### Extracting Vital Signs using YOLOv5m6

### Text extraction:
- Once the outputs from YOLO are obtained, the only task left is to recognize the numbers within the bounding boxes.

- We first implemented EasyOCR, a built-in python library which gave quite unsatisfactory results, so we trained and tested TesseractOCR. The problem with Tesseract is that it has too many modes,making it difficult to identify small and blurry text.

- **PaddlePaddle** has many benefits over the other architectures mentioned above. PaddlePaddle OCR has state-of-the-art recognition accuracy. It works better than Tesseract when images are in RGB **/** BGR if the image cannot be binarised.It is based on the PaddlePaddle Platform, which is extremely fast and makes our model useful for real time application.

- We first convert the RGB image to Grayscale. To adjust for very small bounding boxes generated by YOLO during Vital Sign Extraction, we added a padding of 50 pixels. This made it possible for PaddlePaddleOCR to detect the text easily.

### H.R Curve Detection and Digitisation :
- The task of extracting the curve from the image surprisingly turned out to be more complicated than anticipated. The bounding boxes for the HR graphs contain a lot of text on and around the actual curve making it pretty hard to extract out only the curve.

- Performing edge detection and contour tracing on the image did not give satisfactory results.

- We then tried to devise an **algorithm** : a kernel would scan the surrounding pixels to the right and follow the direction of the highest intensity pixels, thus essentially scanning a single pixel thick heart rate curve. However, this algorithm had too big of a time complexity to be used in real time and the problem of overlapping text and discontinuity in the curve made the problem too complex.

- **Finally** we built a network that consists of a noise reducing-network followed by a series of steps to convert the image of the curve to a plot.

1. First we train a YOLOv5s model to learn any noise in the image. For this we manually annotated the H.R. curves with bounding boxes containing any noise near the curve .

2. We use the predictions by the aforementioned YOLO network to remove the garbage portions of the image(noise and unnecessary text) by blacking out all the predicted bounding boxes(changing pixel values to 0).

3. We then binarize the image, and use contour plotting.

4. Finally we sample the obtained image to get a series of points that is displayed as a plot.

### Hyperparameters:

**Yolov5** :

- Adam optimiser with a weight decay of 0.0005 and momentum of 0.937.
- initial learning rate of 0.01 and final OneCycleLR learning rate of 0.01.
- We used the cosine LR scheduler with maximum learning rate set to the initial learning rate of 0.01 and T\_max set to 50.
- Our batch size was 32 for the entire pipeline.

# Results
<br> </br>
![loss_curves](./images/loss_curves.jpeg)
<br> </br>
| **Technique** | **Accuracy (in %)** | **Inference time per image (in seconds)** |
| --- | --- | --- |
| Screen Extraction with Mask-RCNN → single YOLOv5 network for vitals  | 90.21 | 11.57  |
| **Screen Extraction with YOLOv5m6 → 2 separate YOLOv5 networks for vitals** | **93.723** | **2.26**  |

<br>

Our final pipeline gave a 93.723% accuracy when tested on a randomly chosen set of 1000 images from the given unlabelled dataset, which was manually annotated. The model had an average inference time of 2.26 seconds per image(a 412% increase in speed from the Mask-RCNN based pipeline) 

## References:
[Mask -RCNN](https://arxiv.org/abs/1703.06870)
<br> </br>
[YOLO](https://arxiv.org/abs/1506.02640)
<br> </br>
[PaddlePaddle OCR](https://arxiv.org/abs/2009.09941)
<br> </br>
[Resnet](https://arxiv.org/abs/1512.03385)
<br> </br>
[An Overview of Deep Semi-Supervised Learning](https://arxiv.org/abs/2006.05278)
